In [4]:
import json
import requests
import string
import re
import nltk
import string
import itertools
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
import pke
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import traceback
from nltk.tokenize import sent_tokenize
from flashtext import KeywordProcessor
import textwrap
from IPython.core.display import display, HTML
import xml.etree.ElementTree as et
import random


Downloading:  10%|▉         | 115M/1.14G [03:33<32:32, 564kB/s] 
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yaash\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yaash\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yaash\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
C:\Users\yaash\AppData\Local\Temp\ipykernel_31604\746595572.py:18: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Download the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")
model = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-cnn-12-6")

# Save the tokenizer and model files to a local directory
save_directory = "summarizer/"
tokenizer.save_pretrained("summarizer/tokenizer")
model.save_pretrained("summarizer/model")


Downloading: 100%|██████████| 1.14G/1.14G [01:51<00:00, 11.0MB/s]


In [11]:
inputs = tokenizer(text, max_length=1024, return_tensors="pt")

# Generate Summary
summary_ids = model.generate(inputs["input_ids"], num_beams=2, min_length=0)
tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]


' Humans have long been captivated by the mysteries of outer space . Astronomers like Ptolemy and Galileo have long explored the cosmos . In the 20th century, the space race between the U.S. and the Soviet Union marked a pivotal moment in human history .'

In [13]:
print(tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0])

 Humans have long been captivated by the mysteries of outer space . Astronomers like Ptolemy and Galileo have long explored the cosmos . In the 20th century, the space race between the U.S. and the Soviet Union marked a pivotal moment in human history .


In [6]:
text = """In the vast expanse of the universe, humans have long been captivated by the mysteries of outer space. This journey of exploration and discovery began with ancient astronomers like Ptolemy, who studied celestial bodies and developed the first theories about the Earth's position in the solar system. 

The invention of the telescope in the early 17th century allowed for more detailed observations of the stars and planets, leading to groundbreaking discoveries like Galileo's observation of Jupiter's moons and Saturn's rings. In the 20th century, the space race between the U.S. and the Soviet Union marked a pivotal moment in human history. 

In 1957, the Soviets launched Sputnik 1, the first artificial satellite, which was followed by the United States' Explorer 1 in 1958. Yuri Gagarin, a Soviet cosmonaut, became the first human to journey into outer space in 1961, and in 1969, American astronaut Neil Armstrong made history by becoming the first person to set foot on the Moon as part of the Apollo 11 mission. 

Since then, space exploration has continued to expand, with the development of the International Space Station (ISS), robotic missions to Mars, and ambitious plans for future manned missions to the Moon, Mars, and beyond. As we learn more about our universe, the contributions of countless scientists, astronauts, and engineers serve as a testament to humanity's drive to uncover the secrets of the cosmos."""
wrapper = textwrap.TextWrapper(width=150)
word_list = wrapper.wrap(text=text)
for element in word_list: 
  print(element) 

In the vast expanse of the universe, humans have long been captivated by the mysteries of outer space. This journey of exploration and discovery began
with ancient astronomers like Ptolemy, who studied celestial bodies and developed the first theories about the Earth's position in the solar system.
The invention of the telescope in the early 17th century allowed for more detailed observations of the stars and planets, leading to groundbreaking
discoveries like Galileo's observation of Jupiter's moons and Saturn's rings. In the 20th century, the space race between the U.S. and the Soviet
Union marked a pivotal moment in human history.   In 1957, the Soviets launched Sputnik 1, the first artificial satellite, which was followed by the
United States' Explorer 1 in 1958. Yuri Gagarin, a Soviet cosmonaut, became the first human to journey into outer space in 1961, and in 1969, American
astronaut Neil Armstrong made history by becoming the first person to set foot on the Moon as part of the 

In [25]:
def tokenize_sentences(text):
    sentences = sent_tokenize(text)
    sentences = [sentence.strip() for sentence in sentences if len(sentence) > 20]
    return sentences

sentences = tokenize_sentences(text)
print (sentences)

['Python is a programming language that was created in the late 1980s by Guido van Rossum.', 'Its design philosophy emphasizes code readability and simplicity, which means that Python code is easy to read and write.', 'This makes it a great language for beginners who want to work more efficiently.', 'Python is a versatile, so it can be used for many tasks.', "It's commonly used in web development to build websites and web applications.", "It's also popular in scientific computing, where it's used to analyze and manipulate large datasets.", "In addition, Python is commonly used in artificial intelligence and machine learning, where it's used to develop intelligent systems that can learn and make decisions based on data.", 'One of the reasons why Python is so popular is because it has a large and active community.', 'There are many libraries and tools available that can help you get started with Python and accomplish your goals more quickly.', "Additionally, Python is an open-source lang

In [26]:
def get_noun_adj_verb(text):
    out=[]
    try:
        extractor = pke.unsupervised.MultipartiteRank()
        extractor.load_document(input=text,language='en')
        pos = {'VERB', 'ADJ', 'NOUN'}
        stoplist = list(string.punctuation)
        stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
        stoplist += stopwords.words('english')
        extractor.candidate_selection(pos=pos)
        # 4. build the Multipartite graph and rank candidates using random walk,
        #    alpha controls the weight adjustment mechanism, see TopicRank for
        #    threshold/method parameters.
        extractor.candidate_weighting(alpha=1.1,
                                      threshold=0.75,
                                      method='average')
        keyphrases = extractor.get_n_best(n=30)
        for val in keyphrases:
            out.append(val[0])
    except:
        out = []
        traceback.print_exc()
    return out
noun_verbs_adj = get_noun_adj_verb(text)
print ("keywords: ",noun_verbs_adj)

keywords:  ['programming language', 'machine learning', 'build websites', 'artificial intelligence', 'easy', 'manipulate large datasets', 'popular', 'means', 'web development', 'versatile', 'learn', 'accomplish', 'created', 'help', 'web applications', 'goals', 'read', 'analyze', 'beginners', 'write', 'code', 'want', 'late 1980s', 'design philosophy emphasizes code readability', 'simplicity', 'makes', 'scientific computing', 'great language', 'addition', 'work']


In [27]:
def get_sentences_for_keyword(keywords, sentences):
    keyword_processor = KeywordProcessor()
    keyword_sentences = {}
    for word in keywords:
        keyword_sentences[word] = []
        keyword_processor.add_keyword(word)
    for sentence in sentences:
        keywords_found = keyword_processor.extract_keywords(sentence)
        for key in keywords_found:
            keyword_sentences[key].append(sentence)

    for key in keyword_sentences.keys():
        values = keyword_sentences[key]
        values = sorted(values, key=len, reverse=True)
        keyword_sentences[key] = values
    return keyword_sentences

keyword_sentence_mapping_noun_verbs_adj = get_sentences_for_keyword(noun_verbs_adj, sentences)
def get_fill_in_the_blanks(sentence_mapping):
    out={}
    blank_sentences = []
    processed = []
    keys=[]
    for key in sentence_mapping:
        if len(sentence_mapping[key])>0:
            sent = sentence_mapping[key][0]
            # Compile a regular expression pattern into a regular expression object, which can be used for matching and other methods
            insensitive_sent = re.compile(re.escape(key), re.IGNORECASE)
            no_of_replacements =  len(re.findall(re.escape(key),sent,re.IGNORECASE))
            line = insensitive_sent.sub(' _________ ', sent)
            if (sentence_mapping[key][0] not in processed) and no_of_replacements<2:
                blank_sentences.append(line)
                processed.append(sentence_mapping[key][0])
                keys.append(key)
    out["sentences"]=blank_sentences[:10]
    out["keys"]=keys[:10]
    return out


fill_in_the_blanks = get_fill_in_the_blanks(keyword_sentence_mapping_noun_verbs_adj)
print(fill_in_the_blanks)

{'sentences': ['Python is a  _________  that was created in the late 1980s by Guido van Rossum.', "In addition, Python is commonly used in artificial intelligence and  _________ , where it's used to develop intelligent systems that can learn and make decisions based on data.", "It's commonly used in web development to  _________  and web applications.", 'Its design philosophy emphasizes code readability and simplicity, which means that Python code is  _________  to read and write.', "It's also popular in scientific computing, where it's used to analyze and  _________ .", 'Overall, Python is a powerful and  _________  and is easy to learn and use.', "Whether you're just starting out or you're an experienced developer, Python can help you build a wide variety of projects and  _________  your goals more quickly and efficiently.”", 'This makes it a great language for  _________  who want to work more efficiently.'], 'keys': ['programming language', 'machine learning', 'build websites', 'ea

In [33]:
output_json = [
    {"question": sentence, "answer": key}
    for sentence, key in zip(fill_in_the_blanks["sentences"], fill_in_the_blanks["keys"])
]
print(json.dumps(output_json, indent=2))

[
  {
    "question": "Python is a  _________  that was created in the late 1980s by Guido van Rossum.",
    "answer": "programming language"
  },
  {
    "question": "In addition, Python is commonly used in artificial intelligence and  _________ , where it's used to develop intelligent systems that can learn and make decisions based on data.",
    "answer": "machine learning"
  },
  {
    "question": "It's commonly used in web development to  _________  and web applications.",
    "answer": "build websites"
  },
  {
    "question": "Its design philosophy emphasizes code readability and simplicity, which means that Python code is  _________  to read and write.",
    "answer": "easy"
  },
  {
    "question": "It's also popular in scientific computing, where it's used to analyze and  _________ .",
    "answer": "manipulate large datasets"
  },
  {
    "question": "Overall, Python is a powerful and  _________  and is easy to learn and use.",
    "answer": "versatile"
  },
  {
    "questio

In [34]:

root = et.Element("div")

heading = et.Element("h2")

keywords = et.Element("ul")
keywords.set('style', 'color:blue;')

all_keys = fill_in_the_blanks['keys']
for blank in all_keys:
  child=et.Element("li")
  child.text = blank
  keywords.append(child)

sentences = et.Element("ol")
sentences.set('style', 'color:brown;')
for sentence in fill_in_the_blanks['sentences']:
  child=et.Element("li")
  child.text = sentence
  sentences.append(child)
  sentences.append(et.Element("br"))

heading_content = et.Element("h4")

root.append(heading)
heading_content.append(keywords)
heading_content.append(sentences)
root.append(heading_content)

xmlstr = et.tostring(root)
xmlstr = xmlstr.decode("utf-8") 
display(HTML(xmlstr))